In [ ]:
# CUDA compiler 버전 확인
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
# Jupyter Notebook에서 CUDA 코드 실행하게 해주는 도구 다운로드
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-se8190xg
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-se8190xg
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-_osl851a/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
# Load nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp3xvqbwqi".


In [ ]:
# Serial Hello_World
%%cuda
#include <stdio.h>

int main( void ) {
    printf( "Hello, World! \n" );
    return 0;
}

Hello, World! 



In [ ]:
# CUDA Hello_World
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<2, 2>>>();
    cudaDeviceSynchronize();
}

Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1



In [ ]:
# CUDA Vector Addition
%%cuda
#include <stdio.h>
#include <cuda.h>
__global__ void matrixAddKernel(int *a,int *b, int *c, int N)
{
  int col = threadIdx.x + blockDim.x * blockIdx.x;
  int row = threadIdx.y + blockDim.y * blockIdx.y;
  int index = row * N + col;
  if(col < N && row < N)
  {
    c[index] = a[index]+b[index];
  }
}

void matrixAdd(int *a, int *b, int *c, int N)
{
  int index;
  for(int col=0; col<N; col++)
  {
    for(int row=0; row<N; row++)
    {
      index = row * N + col;
      c[index] = a[index] + b[index];
    }
  }
}

int main(int argc, char *argv[])
{
  //matrix size in each dimension
  int N = 100;
  //grid and block sizes
  dim3 grid(1, 1, 1);
  dim3 block(10000, 1, 1);
  //host memory pointers
  int *a_h;
  int *b_h;
  int *c_h;
  int *d_h;
  //device memory pointers
  int *a_d;
  int *b_d;
  int *c_d;
  //number of bytes in arrays
  int size;
  //variable used for storing keyboard input
  char key;
  //CUDA events to measure time
  cudaEvent_t start;
  cudaEvent_t stop;
  float elapsedTime;
  //print out summary
  printf("Number of threads: %i (%ix%i)\n", block.x*block.y,block.x, block.y);
  printf("Number of blocks: %i (%ix%i)\n", grid.x*grid.y, grid.x,grid.y);
  //number of bytes in each array
  size = N * N * sizeof(int);
  //allocate memory on host, this time we are using dynamic
  //allocation
  a_h = (int*) malloc(size);
  b_h = (int*) malloc(size);
  c_h = (int*) malloc(size);
  d_h = (int*) malloc(size);
  //load arrays with some numbers
  for(int i=0; i<N; i++)
  {
    for(int j=0; j<N; j++)
    {
      a_h[i * N + j] = i;
      b_h[i * N + j] = i;
    }
  }
  //GPU computation//////////////////////////////////
  //allocate device memory
  cudaMalloc((void**)&a_d, size);
  cudaMalloc((void**)&b_d, size);
  cudaMalloc((void**)&c_d, size);
  //copy the host arrays to device
  cudaMemcpy(a_d, a_h, size, cudaMemcpyHostToDevice);
  cudaMemcpy(b_d, b_h, size, cudaMemcpyHostToDevice);
  cudaMemcpy(c_d, c_h, size, cudaMemcpyHostToDevice);
  //start timer
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  cudaEventRecord(start, 0);
  //launch kernel
  matrixAddKernel<<<grid, block>>>(a_d, b_d, c_d, N);
  //stop timer
  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start, stop);
  //print out execution time
  printf("Time to calculate results on GPU: %f ms.\n", elapsedTime);
  //copy the results to host
  cudaMemcpy(c_h, c_d, size ,cudaMemcpyDeviceToHost);
  //grid and block sizes
  //CPU computation//////////////////////////////////
  //start timer
  cudaEventRecord(start, 0);
  //do the calculation on host
  matrixAdd(a_h, b_h, d_h, N);
  //stop timer
  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start, stop );
  //print out execution time
  printf("Time to calculate results on CPU: %f ms.\n", elapsedTime);
  //check if the CPU and GPU results match
  for(int i=0; i<N*N; i++)
  {
    if (c_h[i] != d_h[i]) printf("Error: CPU and GPU results do not match\n");
    break;
  }
  //clean up
  free(a_h);
  free(b_h);
  free(c_h);
  free(d_h);
  cudaFree(a_d);
  cudaFree(b_d);
  cudaFree(c_d);
  cudaEventDestroy(start);
  cudaEventDestroy(stop);

  return 0;
}

Number of threads: 10000 (10000x1)
Number of blocks: 1 (1x1)
Time to calculate results on GPU: 0.165952 ms.
Time to calculate results on CPU: 0.084800 ms.



In [9]:
# 헤더파일을 구글 클라우드에 올리기 위해 현재 폴더 위치 확인
!pwd

/content


In [10]:
# 아래와 같이 Google drive에 접속 후, 드라이브 이동하여 "Colb Notebooks" 폴더 생성 후 헤더파일들이 들어있는 common 폴더 업로드
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%cuda
#include "/content/drive/MyDrive/Colab Notebooks/common/book.h"

#define N   1000

__global__ void add( int *a, int *b, int *c ) {
    int tid = blockIdx.x;    // this thread handles the data at its thread id
    if (tid < N)
        c[tid] = a[tid] + b[tid];
}

int main( void ) {
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;

    // allocate the memory on the GPU
    HANDLE_ERROR( cudaMalloc( (void**)&dev_a, N * sizeof(int) ) );
    HANDLE_ERROR( cudaMalloc( (void**)&dev_b, N * sizeof(int) ) );
    HANDLE_ERROR( cudaMalloc( (void**)&dev_c, N * sizeof(int) ) );

    // fill the arrays 'a' and 'b' on the CPU
    for (int i=0; i<N; i++) {
        a[i] = -i;
        b[i] = i * i;
    }

    // copy the arrays 'a' and 'b' to the GPU
    HANDLE_ERROR( cudaMemcpy( dev_a, a, N * sizeof(int),
                              cudaMemcpyHostToDevice ) );
    HANDLE_ERROR( cudaMemcpy( dev_b, b, N * sizeof(int),
                              cudaMemcpyHostToDevice ) );

    add<<<N,1>>>( dev_a, dev_b, dev_c );

    // copy the array 'c' back from the GPU to the CPU
    HANDLE_ERROR( cudaMemcpy( c, dev_c, N * sizeof(int),
                              cudaMemcpyDeviceToHost ) );

    // display the results
    for (int i=0; i<N; i++) {
        printf( "%d + %d = %d\n", a[i], b[i], c[i] );
    }

    // free the memory allocated on the GPU
    HANDLE_ERROR( cudaFree( dev_a ) );
    HANDLE_ERROR( cudaFree( dev_b ) );
    HANDLE_ERROR( cudaFree( dev_c ) );

    return 0;
}

0 + 0 = 0
-1 + 1 = 0
-2 + 4 = 2
-3 + 9 = 6
-4 + 16 = 12
-5 + 25 = 20
-6 + 36 = 30
-7 + 49 = 42
-8 + 64 = 56
-9 + 81 = 72
-10 + 100 = 90
-11 + 121 = 110
-12 + 144 = 132
-13 + 169 = 156
-14 + 196 = 182
-15 + 225 = 210
-16 + 256 = 240
-17 + 289 = 272
-18 + 324 = 306
-19 + 361 = 342
-20 + 400 = 380
-21 + 441 = 420
-22 + 484 = 462
-23 + 529 = 506
-24 + 576 = 552
-25 + 625 = 600
-26 + 676 = 650
-27 + 729 = 702
-28 + 784 = 756
-29 + 841 = 812
-30 + 900 = 870
-31 + 961 = 930
-32 + 1024 = 992
-33 + 1089 = 1056
-34 + 1156 = 1122
-35 + 1225 = 1190
-36 + 1296 = 1260
-37 + 1369 = 1332
-38 + 1444 = 1406
-39 + 1521 = 1482
-40 + 1600 = 1560
-41 + 1681 = 1640
-42 + 1764 = 1722
-43 + 1849 = 1806
-44 + 1936 = 1892
-45 + 2025 = 1980
-46 + 2116 = 2070
-47 + 2209 = 2162
-48 + 2304 = 2256
-49 + 2401 = 2352
-50 + 2500 = 2450
-51 + 2601 = 2550
-52 + 2704 = 2652
-53 + 2809 = 2756
-54 + 2916 = 2862
-55 + 3025 = 2970
-56 + 3136 = 3080
-57 + 3249 = 3192
-58 + 3364 = 3306
-59 + 3481 = 3422
-60 + 3600 = 3540
-61 + 3

In [ ]:
# 추가 벡터합 예제: CUDA 이벤트 API로 시간 측정
%%cuda
#include <stdio.h>
#include <cuda.h>

// Array 크기 지정
#define N 10  //1024   //4096

// 벡터합 커널 kernel
__global__ void vectorAddKernel(int *a, int *b, int *c)
{
  int tdx = blockIdx.x * blockDim.x + threadIdx.x;
  if(tdx < N)
  {
    c[tdx] = a[tdx]+b[tdx];
  }
}

int main()
{
  // 그리드와 블록 크기 grid and block sizes
  dim3 grid(1, 1, 1);
  dim3 block(1024, 1, 1);

  //host arrays
  int a_h[N];
  int b_h[N];
  int c_h[N];

  // 디바이스 메모리 포인터 device memory pointers
  int *a_d;
  int *b_d;
  int *c_d;

  // 배열 초기값 지정 load arrays with some numbers
  for(int i=0; i<N; i++)
  {
    a_h[i] = i;
    b_h[i] = i*1;
  }

  // 디바이스 메모리 할당 allocate device memory
  cudaMalloc((void**)&a_d, N*sizeof(int));
  cudaMalloc((void**)&b_d, N*sizeof(int));
  cudaMalloc((void**)&c_d, N*sizeof(int));

  // 호스트 배열을 디바이스로 복사 copy the host arrays to device
  cudaMemcpy(a_d, a_h, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(b_d, b_h, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(c_d, c_h, N*sizeof(int), cudaMemcpyHostToDevice);

  // 시간측정 API - CUDA events to measure time
  cudaEvent_t start;
  cudaEvent_t stop;
  float elapsedTime;

  //start timer
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  cudaEventRecord(start, 0);

  //launch kernel
  vectorAddKernel<<<grid, block>>>(a_d, b_d, c_d);

  //stop timer
  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start, stop);

  //copy the results to host
  cudaMemcpy(c_h, c_d, N*sizeof(int), cudaMemcpyDeviceToHost);

  //print the results
  for(int i=0; i<N; i++)
  {
    printf("%i+%i = %i\n",a_h[i], b_h[i], c_h[i]);
  }

  //print out execution time
  printf("Time to calculate results: %f ms.\n", elapsedTime);

  //clean up
  cudaFree(a_h);
  cudaFree(b_h);
  cudaFree(c_h);
  cudaEventDestroy(start);
  cudaEventDestroy(stop);

  return 0;
}

0+0 = 0
1+1 = 2
2+2 = 4
3+3 = 6
4+4 = 8
5+5 = 10
6+6 = 12
7+7 = 14
8+8 = 16
9+9 = 18
Time to calculate results: 0.222784 ms.



In [12]:
# Dot Product
%%cuda
#include "/content/drive/MyDrive/Colab Notebooks/common/book.h"

#define imin(a,b) (a<b?a:b)

const int N = 33 * 1024;
const int threadsPerBlock = 256;
const int blocksPerGrid =
            imin( 32, (N+threadsPerBlock-1) / threadsPerBlock );


__global__ void dot( float *a, float *b, float *c ) {
    __shared__ float cache[threadsPerBlock];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int cacheIndex = threadIdx.x;

    float   temp = 0;
    while (tid < N) {
        temp += a[tid] * b[tid];
        tid += blockDim.x * gridDim.x;
    }

    // set the cache values
    cache[cacheIndex] = temp;

    // synchronize threads in this block
    __syncthreads();

    // for reductions, threadsPerBlock must be a power of 2
    // because of the following code
    int i = blockDim.x/2;
    while (i != 0) {
        if (cacheIndex < i)
            cache[cacheIndex] += cache[cacheIndex + i];
        __syncthreads();
        i /= 2;
    }

    if (cacheIndex == 0)
        c[blockIdx.x] = cache[0];
}


int main( void ) {
    float   *a, *b, c, *partial_c;
    float   *dev_a, *dev_b, *dev_partial_c;

    // allocate memory on the cpu side
    a = (float*)malloc( N*sizeof(float) );
    b = (float*)malloc( N*sizeof(float) );
    partial_c = (float*)malloc( blocksPerGrid*sizeof(float) );

    // allocate the memory on the GPU
    HANDLE_ERROR( cudaMalloc( (void**)&dev_a,
                              N*sizeof(float) ) );
    HANDLE_ERROR( cudaMalloc( (void**)&dev_b,
                              N*sizeof(float) ) );
    HANDLE_ERROR( cudaMalloc( (void**)&dev_partial_c,
                              blocksPerGrid*sizeof(float) ) );

    // fill in the host memory with data
    for (int i=0; i<N; i++) {
        a[i] = i;
        b[i] = i*2;
    }

    // copy the arrays 'a' and 'b' to the GPU
    HANDLE_ERROR( cudaMemcpy( dev_a, a, N*sizeof(float),
                              cudaMemcpyHostToDevice ) );
    HANDLE_ERROR( cudaMemcpy( dev_b, b, N*sizeof(float),
                              cudaMemcpyHostToDevice ) );

    dot<<<blocksPerGrid,threadsPerBlock>>>( dev_a, dev_b,
                                            dev_partial_c );

    // copy the array 'c' back from the GPU to the CPU
    HANDLE_ERROR( cudaMemcpy( partial_c, dev_partial_c,
                              blocksPerGrid*sizeof(float),
                              cudaMemcpyDeviceToHost ) );

    // finish up on the CPU side
    c = 0;
    for (int i=0; i<blocksPerGrid; i++) {
        c += partial_c[i];
    }

    #define sum_squares(x)  (x*(x+1)*(2*x+1)/6)
    printf( "Does GPU value %.6g = %.6g?\n", c,
             2 * sum_squares( (float)(N - 1) ) );

    // free memory on the gpu side
    HANDLE_ERROR( cudaFree( dev_a ) );
    HANDLE_ERROR( cudaFree( dev_b ) );
    HANDLE_ERROR( cudaFree( dev_partial_c ) );

    // free memory on the cpu side
    free( a );
    free( b );
    free( partial_c );
}


Does GPU value 2.57236e+13 = 2.57236e+13?

